<a href="https://colab.research.google.com/github/kazeidk/GDrive_Turbo_Copy/blob/main/GDrive_Turbo_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 GDrive Turbo Copy v1.2
**⚡ Copy Google Drive → Google Drive | Nhanh • Ổn định • ♾️ KHÔNG GIỚI HẠN**

👨‍💻 **Nguyễn Ngọc Anh Tú** | 📢 [Kênh TB](https://www.messenger.com/channel/NguyenNgocAnhTu.VN) • [FB](https://www.facebook.com/NguyenNgocAnhTu.VN) • [TG](https://t.me/NguyenNgocAnhTu) • [GitHub](https://github.com/kazeidk)

---
♾️ **Không giới hạn dung lượng** • ⚡ Copy trực tiếp • 🔄 Auto-resume • 💾 Checkpoint • 📄 Export Docs→PDF

**📖 Dùng:** `Cell 1` Nhập link → `Cell 2` Copy → Timeout/Limit? Chạy lại Cell 2 (hoặc đổi account)

**💡 Tips:** 🌙 Chạy đêm ít rate limit • 💪 Colab Pro + High RAM • 📋 Log: `/content/gdrive_copy.log`

In [ ]:
#@title 1️⃣ 📝 Nhập thông tin
from ipywidgets import widgets, Layout
from IPython.display import display, HTML

style = {'description_width': '140px'}
layout = Layout(width='580px')

display(HTML('''
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 12px; margin-bottom: 20px; box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
    <h2 style="color: white; margin: 0; font-size: 24px;">🚀 GDrive Turbo Copy v1.2</h2>
    <p style="color: #e0e0e0; margin: 8px 0 0 0; font-size: 14px;">⚡ Copy Google Drive siêu nhanh • ♾️ Không giới hạn dung lượng</p>
</div>
'''))

dest_text = widgets.Text(description='📁 Folder đích:', placeholder='🔗 Paste link folder Google Drive của bạn', style=style, layout=layout)
source_text = widgets.Text(description='📂 Folder nguồn:', placeholder='🔗 Paste link folder cần copy (Shared Drive ✓)', style=style, layout=layout)

display(HTML('<h4>📌 Thông tin bắt buộc:</h4>'))
display(dest_text, source_text)

exclude_str_text = widgets.Text(description='🚫 Bỏ qua chứa:', value='', placeholder='📝 VD: .tmp, backup, test', style=style, layout=layout)
include_ext_text = widgets.Text(description='✅ Chỉ lấy đuôi:', value='', placeholder='📝 VD: .mp4, .pdf, .zip', style=style, layout=layout)
exclude_ext_text = widgets.Text(description='❌ Bỏ qua đuôi:', value='', placeholder='📝 VD: .tmp, .log, .bak', style=style, layout=layout)

display(HTML('<h4>🔍 Bộ lọc (tùy chọn):</h4>'))
display(exclude_str_text, include_ext_text, exclude_ext_text)

skip_exists_cb = widgets.Checkbox(description='⏭️ Bỏ qua file đã có (khuyến nghị ✓)', value=True, indent=False, layout=Layout(width='320px'))
export_gdocs_cb = widgets.Checkbox(description='📄 Export Google Docs → PDF', value=False, indent=False, layout=Layout(width='320px'))
dry_run_cb = widgets.Checkbox(description='👁️ Dry-run (chỉ xem, không copy)', value=False, indent=False, layout=Layout(width='320px'))
skip_verify_cb = widgets.Checkbox(description='⏩ Bỏ qua verify (nhanh hơn)', value=False, indent=False, layout=Layout(width='320px'))

display(HTML('<h4>⚙️ Cài đặt:</h4>'))
display(skip_exists_cb, export_gdocs_cb, dry_run_cb, skip_verify_cb)

display(HTML('''
<div style="background: linear-gradient(135deg, #28a745 0%, #20c997 100%); padding: 12px 15px; border-radius: 8px; margin-top: 20px; color: white; box-shadow: 0 2px 10px rgba(40,167,69,0.3);">
    <b>✅ Nhập xong?</b> → Chạy <b>Cell 2</b> để bắt đầu copy! 🚀
</div>
'''))

In [ ]:
#@title 2️⃣ 🚀 Run - GDrive Turbo Copy
import os, time, re, json, logging, warnings, gc, io, uuid, socket
from datetime import datetime
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload
from google.colab import auth
from IPython.display import Audio, display, HTML, clear_output
import numpy as np
import ipywidgets as widgets

warnings.filterwarnings('ignore')
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)
logging.getLogger('google_auth_httplib2').setLevel(logging.ERROR)

class GDriveTurboCopy:
    VERSION = '1.2'
    CHECKPOINT_NAME = '.gdrive_turbo_checkpoint.json'
    EXPORT_MIMES = {
        'application/vnd.google-apps.document': ('application/pdf', '.pdf'),
        'application/vnd.google-apps.spreadsheet': ('application/pdf', '.pdf'),
        'application/vnd.google-apps.presentation': ('application/pdf', '.pdf'),
        'application/vnd.google-apps.drawing': ('application/pdf', '.pdf'),
    }
    API_TIMEOUT = 300
    MAX_CACHE_SIZE = 10000

    def __init__(self):
        self.total_mb = self.copied = self.skipped = self.errors = self.exported = 0
        self.files_this_session = 0
        self.skip_verify = False
        self.error_files, self.shortcuts, self.done_keys = [], [], set()
        self.exclude_str, self.include_ext, self.exclude_ext = [], [], []
        self.dry_run = False
        self.skip_exists = True
        self.export_gdocs = False
        self.checkpoint_local = '/content/checkpoint.json'
        self.log_file = '/content/gdrive_copy.log'
        self.svc = None
        self.start_time = self.last_gc = self.last_save = 0
        self.source_url = self.dest_url = self.current_path = ''
        self.folder_cache = {}
        self.dest_files_cache = {}
        self.min_delay = 0.1
        self.last_request = 0
        self.rate_limit_hits = 0
        self.dest_id = None
        self.checkpoint_file_id = None
        self.session_id = uuid.uuid4().hex[:8]
        self._init_log()

    def _init_log(self):
        self.logger = logging.getLogger('GDriveCopy')
        self.logger.setLevel(logging.INFO)
        self.logger.handlers = []
        fh = logging.FileHandler(self.log_file, encoding='utf-8')
        fh.setFormatter(logging.Formatter('[%(asctime)s] %(message)s', '%H:%M:%S'))
        self.logger.addHandler(fh)

    def log(self, msg, also_print=True):
        self.logger.info(msg)
        if also_print:
            print(f'[{datetime.now().strftime("%H:%M:%S")}] {msg}')

    def rate_limit(self):
        elapsed = time.time() - self.last_request
        if elapsed < self.min_delay:
            time.sleep(self.min_delay - elapsed)
        self.last_request = time.time()

    def adjust_rate(self, hit_limit=False):
        if hit_limit:
            self.rate_limit_hits += 1
            self.min_delay = min(self.min_delay * 1.5, 2.0)
        elif self.rate_limit_hits > 0 and self.min_delay > 0.1:
            self.min_delay = max(self.min_delay * 0.95, 0.1)

    def clear_cache_if_needed(self):
        if len(self.dest_files_cache) > self.MAX_CACHE_SIZE:
            keys = list(self.dest_files_cache.keys())
            for k in keys[:len(keys)//2]:
                del self.dest_files_cache[k]
            self.log(f'🧹 Đã clear {len(keys)//2} cache entries', also_print=False)
        if len(self.folder_cache) > self.MAX_CACHE_SIZE:
            keys = list(self.folder_cache.keys())
            for k in keys[:len(keys)//2]:
                del self.folder_cache[k]

    def load_cp_from_drive(self):
        if not self.dest_id:
            return False
        try:
            r = self.call(lambda: self.svc.files().list(
                q=f"'{self.dest_id}' in parents and name='{self.CHECKPOINT_NAME}' and trashed=false",
                fields='files(id,modifiedTime)', supportsAllDrives=True, pageSize=1).execute())
            if r and r.get('files'):
                self.checkpoint_file_id = r['files'][0]['id']
                cp_time = r['files'][0].get('modifiedTime', '')[:19].replace('T', ' ')
                req = self.svc.files().get_media(fileId=self.checkpoint_file_id)
                buf = io.BytesIO()
                dl = MediaIoBaseDownload(buf, req)
                done = False
                while not done:
                    _, done = dl.next_chunk()
                buf.seek(0)
                d = json.loads(buf.read().decode('utf-8'))
                self.done_keys = set(d.get('k', []))
                self.total_mb, self.copied, self.skipped = d.get('m', 0), d.get('c', 0), d.get('s', 0)
                self.exported, self.error_files, self.shortcuts = d.get('exp', 0), d.get('e', []), d.get('shortcuts', [])
                if self.done_keys:
                    self.log(f'☁️ RESUME từ checkpoint (lưu: {cp_time})')
                    self.log(f'   📊 Đã xong: {self.copied:,} copied | {self.skipped:,} skipped | {self.total_mb/1024:.2f}GB')
                    self.log(f'   🔄 Tiếp tục copy các file còn lại...')
                else:
                    self.log(f'📋 Checkpoint rỗng, bắt đầu copy mới')
                return True
        except Exception as e:
            self.log(f'⚠️ Không đọc được checkpoint từ Drive: {str(e)[:50]}', also_print=False)
        return False

    def load_cp(self):
        if self.load_cp_from_drive():
            return True
        try:
            if os.path.exists(self.checkpoint_local):
                with open(self.checkpoint_local, 'r') as f:
                    d = json.load(f)
                self.done_keys = set(d.get('k', []))
                self.total_mb, self.copied, self.skipped = d.get('m', 0), d.get('c', 0), d.get('s', 0)
                self.exported, self.error_files, self.shortcuts = d.get('exp', 0), d.get('e', []), d.get('shortcuts', [])
                cp_time = d.get('t', 'N/A')[:19] if d.get('t') else 'N/A'
                if self.done_keys:
                    self.log(f'🔄 RESUME từ local checkpoint (lưu: {cp_time})')
                    self.log(f'   📊 Đã xong: {self.copied:,} copied | {self.skipped:,} skipped | {self.total_mb/1024:.2f}GB')
                    self.log(f'   🔄 Tiếp tục copy các file còn lại...')
                else:
                    self.log(f'📋 Checkpoint rỗng, bắt đầu copy mới')
                return True
        except Exception:
            pass
        return False

    def save_cp_to_drive(self):
        if not self.dest_id:
            return
        try:
            data = {
                'k': list(self.done_keys), 'm': self.total_mb, 'c': self.copied, 's': self.skipped,
                'exp': self.exported, 'e': self.error_files[-100:], 'shortcuts': self.shortcuts[-50:],
                'src': self.source_url, 'dst': self.dest_url, 't': datetime.now().isoformat(),
                'v': self.VERSION, 'session': self.session_id
            }
            content = json.dumps(data).encode('utf-8')
            media = MediaIoBaseUpload(io.BytesIO(content), mimetype='application/json', resumable=False)
            if self.checkpoint_file_id:
                self.call(lambda: self.svc.files().update(fileId=self.checkpoint_file_id, media_body=media).execute())
            else:
                r = self.call(lambda: self.svc.files().create(
                    body={'name': self.CHECKPOINT_NAME, 'parents': [self.dest_id]},
                    media_body=media, fields='id', supportsAllDrives=True).execute())
                if r:
                    self.checkpoint_file_id = r['id']
        except Exception as e:
            self.log(f'⚠️ Không lưu được checkpoint lên Drive: {str(e)[:50]}', also_print=False)

    def save_cp(self, force=False):
        now = time.time()
        if not force and now - self.last_save < 30:
            return
        self.last_save = now
        try:
            data = {
                'k': list(self.done_keys), 'm': self.total_mb, 'c': self.copied, 's': self.skipped,
                'exp': self.exported, 'e': self.error_files[-100:], 'shortcuts': self.shortcuts[-50:],
                'src': self.source_url, 'dst': self.dest_url, 't': datetime.now().isoformat(),
                'v': self.VERSION, 'session': self.session_id
            }
            with open(self.checkpoint_local, 'w') as f:
                json.dump(data, f)
        except Exception:
            pass
        self.save_cp_to_drive()

    def cleanup_checkpoint(self):
        deleted = []
        if os.path.exists(self.checkpoint_local):
            try:
                os.remove(self.checkpoint_local)
                deleted.append('local')
            except Exception:
                pass
        if self.checkpoint_file_id:
            try:
                self.call(lambda: self.svc.files().delete(fileId=self.checkpoint_file_id, supportsAllDrives=True).execute())
                deleted.append('Drive')
            except Exception:
                pass
        if deleted:
            self.log(f"🧹 Đã xóa checkpoint ({', '.join(deleted)}) - Copy hoàn tất!")

    def auth(self):
        self.log('🔐 Đang xác thực...')
        auth.authenticate_user()
        socket.setdefaulttimeout(self.API_TIMEOUT)
        self.svc = build('drive', 'v3')
        self.log('✅ Xác thực OK!')

    def play_sound(self, success=True):
        try:
            sr = 22050
            freqs, durs = ([523, 659, 784], [0.15, 0.15, 0.2]) if success else ([400, 300], [0.2, 0.3])
            sounds = [np.sin(2 * np.pi * f * np.linspace(0, d, int(sr * d))) * np.exp(-3 * np.linspace(0, d, int(sr * d))) for f, d in zip(freqs, durs)]
            display(Audio(np.concatenate(sounds), rate=sr, autoplay=True))
        except Exception:
            pass

    def fmt_time(self, s):
        if s < 60:
            return f'{int(s)}s'
        if s < 3600:
            return f'{int(s//60)}m{int(s%60)}s'
        return f'{int(s//3600)}h{int((s%3600)//60)}m'

    def fmt_size(self, mb):
        if mb < 1024:
            return f'{mb:.1f}MB'
        if mb < 1024*1024:
            return f'{mb/1024:.2f}GB'
        return f'{mb/(1024*1024):.2f}TB'

    def maybe_gc(self):
        if time.time() - self.last_gc > 120:
            gc.collect()
            self.last_gc = time.time()
            self.clear_cache_if_needed()

    def ok_ext(self, name):
        if not name:
            return True
        ext = os.path.splitext(name)[1].lower()
        if self.include_ext:
            return ext in self.include_ext
        if self.exclude_ext:
            return ext not in self.exclude_ext
        return True

    def clean_name(self, name):
        if not name or not name.strip():
            return 'unnamed'
        return name

    def escape_q(self, s):
        return s.replace('\\', '\\\\').replace("'", "\\'")

    def call(self, fn, retry=5):
        for i in range(retry):
            try:
                self.rate_limit()
                result = fn()
                self.adjust_rate(hit_limit=False)
                return result
            except HttpError as e:
                if e.resp.status == 404:
                    return None
                if e.resp.status in [403, 429]:
                    self.adjust_rate(hit_limit=True)
                    wait = min(30 * (2 ** i), 300)
                    print(f'⚠️ Rate limit, đợi {wait}s...')
                    time.sleep(wait)
                elif e.resp.status in [500, 502, 503, 504]:
                    time.sleep(min(5 * (2 ** i), 60))
                else:
                    raise
            except socket.timeout:
                self.log(f'⚠️ Timeout, retry {i+1}/{retry}...', also_print=True)
                time.sleep(min(5 * (2 ** i), 60))
            except Exception as e:
                if 'timeout' in str(e).lower():
                    self.log(f'⚠️ Timeout, retry {i+1}/{retry}...', also_print=True)
                    time.sleep(min(5 * (2 ** i), 60))
                elif i < retry - 1:
                    time.sleep(min(2 * (2 ** i), 30))
                else:
                    raise
        return None

    def ls(self, fid):
        items, token = [], None
        q = f"'{fid}' in parents and trashed=false"
        for s in self.exclude_str:
            q += f" and not name contains '{self.escape_q(s)}'"
        while True:
            current_token = token
            r = self.call(lambda t=current_token: self.svc.files().list(
                q=q, pageToken=t, pageSize=1000,
                fields='files(id,name,mimeType,size),nextPageToken',
                supportsAllDrives=True, includeItemsFromAllDrives=True).execute())
            if not r:
                break
            for f in r.get('files', []):
                mime, name = f.get('mimeType', ''), f.get('name', '')
                if mime == 'application/vnd.google-apps.shortcut':
                    self.shortcuts.append({'name': name, 'id': f['id'], 'path': self.current_path})
                    continue
                is_folder = mime == 'application/vnd.google-apps.folder'
                is_exportable = mime in self.EXPORT_MIMES
                is_gfile = mime.startswith('application/vnd.google-apps.')
                if is_folder or (is_exportable and self.export_gdocs) or (not is_gfile and self.ok_ext(name)):
                    items.append(f)
            token = r.get('nextPageToken')
            if not token:
                break
        return items

    def cache_dest_files(self, did):
        if did in self.dest_files_cache:
            return
        self.dest_files_cache[did] = set()
        token = None
        while True:
            current_token = token
            r = self.call(lambda t=current_token: self.svc.files().list(
                q=f"'{did}' in parents and trashed=false",
                pageToken=t, pageSize=1000,
                fields='files(name),nextPageToken', supportsAllDrives=True).execute())
            if not r:
                break
            for f in r.get('files', []):
                self.dest_files_cache[did].add(f.get('name', ''))
            token = r.get('nextPageToken')
            if not token:
                break

    def exists_fast(self, did, name):
        if not self.skip_exists:
            return False
        if did not in self.dest_files_cache:
            self.cache_dest_files(did)
        return name in self.dest_files_cache.get(did, set())

    def cp_simple(self, did, src):
        fid, name, mime = src['id'], src.get('name', 'unnamed'), src.get('mimeType', '')
        if mime in self.EXPORT_MIMES:
            if self.export_gdocs:
                self.export_doc_simple(did, src)
            return
        fname = self.clean_name(name)
        mb = int(src.get('size', 0)) / (1024*1024)
        key = f"{did}:{fid}"
        if key in self.done_keys:
            self.skipped += 1
            return
        if self.dry_run:
            self.total_mb += mb
            self.copied += 1
            return
        if self.exists_fast(did, fname):
            self.skipped += 1
            self.done_keys.add(key)
            return
        try:
            r = self.call(lambda: self.svc.files().copy(
                body={'parents': [did], 'name': fname},
                fileId=fid, supportsAllDrives=True).execute())
            if r:
                self.total_mb += mb
                self.copied += 1
                self.done_keys.add(key)
                if did in self.dest_files_cache:
                    self.dest_files_cache[did].add(fname)
                self.maybe_gc()
        except Exception as e:
            self.errors += 1
            self.error_files.append({'name': name, 'id': fid, 'error': str(e)[:100], 'path': self.current_path})

    def export_doc_simple(self, did, src):
        fid, name, mime = src['id'], src.get('name', 'unnamed'), src.get('mimeType', '')
        if mime not in self.EXPORT_MIMES:
            return
        exp_mime, ext = self.EXPORT_MIMES[mime]
        fname = self.clean_name(name) + ext
        key = f"{did}:{fid}:exp"
        if key in self.done_keys:
            self.skipped += 1
            return
        if self.dry_run:
            self.exported += 1
            return
        if self.exists_fast(did, fname):
            self.skipped += 1
            self.done_keys.add(key)
            return
        try:
            buf = io.BytesIO()
            self.rate_limit()
            dl = MediaIoBaseDownload(buf, self.svc.files().export_media(fileId=fid, mimeType=exp_mime))
            done = False
            while not done:
                _, done = dl.next_chunk()
            buf.seek(0)
            r = self.call(lambda: self.svc.files().create(
                body={'name': fname, 'parents': [did]},
                media_body=MediaIoBaseUpload(buf, mimetype=exp_mime, resumable=True),
                fields='id,size', supportsAllDrives=True).execute())
            if r:
                mb = int(r.get('size', 0)) / (1024*1024)
                self.total_mb += mb
                self.exported += 1
                self.done_keys.add(key)
                if did in self.dest_files_cache:
                    self.dest_files_cache[did].add(fname)
        except Exception as e:
            self.errors += 1
            self.error_files.append({'name': name, 'id': fid, 'error': f'Export: {str(e)[:80]}', 'path': self.current_path})

    def mkdir(self, did, name):
        name = self.clean_name(name)
        cache_key = f"{did}:{name}"
        if cache_key in self.folder_cache:
            return self.folder_cache[cache_key]
        if self.exists_fast(did, name):
            r = self.call(lambda: self.svc.files().list(
                q=f"'{did}' in parents and name='{self.escape_q(name)}' and mimeType='application/vnd.google-apps.folder' and trashed=false",
                fields='files(id)', supportsAllDrives=True, pageSize=1).execute())
            if r and r.get('files'):
                fid = r['files'][0]['id']
                self.folder_cache[cache_key] = fid
                return fid
        if self.dry_run:
            return f'dry_{name}'
        try:
            r = self.call(lambda: self.svc.files().create(
                body={'name': name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [did]},
                fields='id', supportsAllDrives=True).execute())
            if r:
                self.folder_cache[cache_key] = r['id']
                if did in self.dest_files_cache:
                    self.dest_files_cache[did].add(name)
                return r['id']
        except Exception as e:
            print(f'❌ Mkdir {name}: {e}')
        return None

    def copy_dir(self, did, sid, path='', depth=0):
        self.current_path = path
        items = self.ls(sid)
        if not items:
            return
        folders = [f for f in items if f.get('mimeType') == 'application/vnd.google-apps.folder']
        files = [f for f in items if f.get('mimeType') != 'application/vnd.google-apps.folder']

        if files:
            if self.skip_exists:
                self.cache_dest_files(did)
            folder_name = (path.split('/')[-1] if path else 'Root')[:35]
            regular_files = [f for f in files if f.get('mimeType') not in self.EXPORT_MIMES]
            new_files = [f for f in regular_files if f"{did}:{f['id']}" not in self.done_keys]

            if len(new_files) == 0:
                for f in regular_files:
                    key = f"{did}:{f['id']}"
                    if key in self.done_keys:
                        self.skipped += 1
                    elif self.exists_fast(did, self.clean_name(f.get('name', 'unnamed'))):
                        self.skipped += 1
                        self.done_keys.add(key)
                print(f'📁 {folder_name}: ✅ {len(files)} files đã xong')
            else:
                print(f'\n📁 {folder_name}: {len(files)} files ({len(new_files)} mới), {len(folders)} folders')
                bar_id = f'pbar_{uuid.uuid4().hex[:8]}'
                pbar_html = widgets.HTML(value='')
                display(pbar_html)
                for i, f in enumerate(files):
                    self.cp_simple(did, f)
                    elapsed = time.time() - self.start_time
                    speed = self.total_mb / max(elapsed, 1)
                    pct = ((i+1) / len(files)) * 100
                    self.files_this_session += 1
                    files_remaining = len(files) - (i + 1)
                    if i > 0:
                        time_per_file = elapsed / self.files_this_session
                        eta_folder = time_per_file * files_remaining
                        eta_str = f'⏳ {self.fmt_time(eta_folder)}'
                    else:
                        eta_str = '⏳ --'
                    pbar_html.value = f'''
                    <style>
                    @keyframes shine_{bar_id} {{ 0% {{ background-position: -100px 0; }} 100% {{ background-position: 100px 0; }} }}
                    .pbar_{bar_id} {{ width: 280px; height: 16px; background: #2d2d2d; border-radius: 8px; overflow: hidden; position: relative; display: inline-block; }}
                    .pbar_{bar_id} .fill {{ height: 100%; background: linear-gradient(90deg, #00d26a, #00ff88, #00d26a); background-size: 100px 100%; border-radius: 9px; transition: width 0.2s; animation: shine_{bar_id} 1s infinite linear; }}
                    .pbar_{bar_id} .pct {{ position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%); color: white; font-weight: bold; font-size: 11px; }}
                    </style>
                    <div class="pbar_{bar_id}"><div class="fill" style="width:{pct}%"></div><div class="pct">{pct:.0f}%</div></div>
                    <div style="font-family:monospace;font-size:12px;margin-top:4px;">⚡ {i+1}/{len(files)} | 💾 {self.fmt_size(self.total_mb)} | 📊 {self.files_this_session*60/max(elapsed,1):.0f}f/m | 🚀 {speed:.1f}MB/s | {eta_str}</div>
                    '''
                self.save_cp()

        for folder in folders:
            fname = folder.get('name', 'unnamed')
            new_path = f'{path}/{fname}' if path else fname
            sub = self.mkdir(did, fname)
            if sub and not sub.startswith('dry_'):
                self.copy_dir(sub, folder['id'], new_path, depth + 1)

    def count_dest_files(self, fid, depth=0):
        count = 0
        token = None
        while True:
            current_token = token
            r = self.call(lambda t=current_token: self.svc.files().list(
                q=f"'{fid}' in parents and trashed=false",
                pageToken=t, pageSize=1000,
                fields='files(id,name,mimeType),nextPageToken',
                supportsAllDrives=True).execute())
            if not r:
                break
            for f in r.get('files', []):
                mime = f.get('mimeType', '')
                name = f.get('name', '')
                if mime == 'application/vnd.google-apps.folder':
                    count += self.count_dest_files(f['id'], depth + 1)
                elif mime == 'application/vnd.google-apps.shortcut':
                    continue
                else:
                    is_exportable = mime in self.EXPORT_MIMES
                    is_gfile = mime.startswith('application/vnd.google-apps.')
                    skip_by_str = any(s in name for s in self.exclude_str) if self.exclude_str else False
                    if skip_by_str:
                        continue
                    if is_gfile and not is_exportable:
                        continue
                    if is_exportable and not self.export_gdocs:
                        continue
                    if not is_gfile and not self.ok_ext(name):
                        continue
                    count += 1
            token = r.get('nextPageToken')
            if not token:
                break
        return count

    def count_src_files(self, fid, depth=0):
        count = 0
        items = self.ls(fid)
        for f in items:
            if f.get('mimeType') == 'application/vnd.google-apps.folder':
                count += self.count_src_files(f['id'], depth + 1)
            else:
                count += 1
        return count

    def verify(self, src_id, dest_id):
        if dest_id is None:
            self.log('⚠️ VERIFY: Không có folder đích để verify')
            return False
        self.log('🔍 Đang verify...')
        src_count = self.count_src_files(src_id)
        dest_count = self.count_dest_files(dest_id)
        if src_count == dest_count:
            self.log(f'✅ VERIFY OK: {src_count:,} files')
            return True
        else:
            diff = src_count - dest_count
            self.log(f'⚠️ VERIFY: Nguồn {src_count:,} | Đích {dest_count:,} | Thiếu {abs(diff):,}')
            self.log(f'   💡 Chạy Cell 2 hoặc Cell 5 (Retry)')
            return False

    def get_id(self, url):
        if not url:
            return None
        m = re.search(r'[-\w]{25,}', url)
        return m.group(0) if m else None

    def summary(self, elapsed):
        speed = self.total_mb / max(elapsed, 1)
        quota_pct = self.total_mb / 1024 / 750 * 100
        print(f'\n{"="*50}')
        print(f'✅ HOÀN TẤT! (v{self.VERSION})')
        print(f'{"="*50}')
        print(f'📁 Copied: {self.copied:,} | ⏭️ Skipped: {self.skipped:,} | ❌ Errors: {self.errors}')
        print(f'💾 {self.fmt_size(self.total_mb)} (~{quota_pct:.0f}% quota) | ⏱️ {self.fmt_time(elapsed)} | 🚀 {speed:.1f} MB/s')
        if self.exported > 0:
            print(f'📄 Exported: {self.exported:,} Google Docs → PDF')
        if self.errors == 0 and self.copied > 0:
            print(f'🎉 Copy hoàn tất không lỗi!')
        elif self.errors == 0 and self.copied == 0 and self.skipped > 0:
            print(f'✅ Tất cả files đã được copy trước đó!')
        if quota_pct > 90:
            print(f'⚠️ Gần hết quota ({quota_pct:.0f}%)! Đổi account để tiếp tục.')
        if self.error_files:
            print(f'⚠️ {len(self.error_files)} lỗi - chạy lại Cell 2 để retry')
            for ef in self.error_files[:3]:
                print(f'   • {ef.get("name", "?")[:40]}: {ef.get("error", "?")[:30]}')

    def run(self, dest, src):
        self.source_url, self.dest_url = src, dest
        self.start_time = self.last_gc = self.last_save = self.last_request = time.time()
        self.auth()
        did, sid = self.get_id(dest), self.get_id(src)
        self.dest_id = did
        if not did or not sid:
            self.log('❌ URL không hợp lệ!')
            return
        try:
            si = self.svc.files().get(fileId=sid, supportsAllDrives=True, fields='name').execute()
            di = self.svc.files().get(fileId=did, supportsAllDrives=True, fields='name').execute()
        except Exception as e:
            self.log(f'❌ Không truy cập được: {e}')
            return
        src_name, dst_name = si.get('name', '?'), di.get('name', '?')
        self.log(f'📂 Nguồn: {src_name}')
        self.log(f'📁 Đích: {dst_name}')
        has_checkpoint = self.load_cp()
        if has_checkpoint and self.done_keys:
            self.log(f'📊 Checkpoint: {len(self.done_keys):,} files đã xử lý')
        self.log(f'⚡ TURBO MODE: Cache + Optimized')
        self.log(f'♾️ Mode: KHÔNG GIỚI HẠN dung lượng')
        self.log('─' * 40)
        new_dest = self.mkdir(did, si.get('name', 'Copy'))
        if not new_dest:
            self.log('❌ Không tạo được folder!')
            self.play_sound(False)
            return
        self.copy_dir(new_dest, sid, si.get('name', 'Copy'))
        self.save_cp(force=True)
        elapsed = time.time() - self.start_time
        self.summary(elapsed)
        self.play_sound(self.errors == 0)
        if not self.skip_verify:
            self.verify(sid, new_dest)
        else:
            self.log('⏩ Bỏ qua verify')
        if self.errors == 0:
            self.cleanup_checkpoint()
        self.log(f'📝 Log: {self.log_file}')

# ═══════════════════════════════════════════════════════
# 🚀 KHỞI CHẠY
# ═══════════════════════════════════════════════════════
from IPython.display import clear_output
clear_output(wait=True)
dl = GDriveTurboCopy()
dl.exclude_str = [s.strip() for s in exclude_str_text.value.split(',') if s.strip()]
dl.include_ext = [('.' + s.strip().lstrip('.').lower()) for s in include_ext_text.value.split(',') if s.strip()]
dl.exclude_ext = [('.' + s.strip().lstrip('.').lower()) for s in exclude_ext_text.value.split(',') if s.strip()]
dl.skip_exists, dl.export_gdocs, dl.dry_run = skip_exists_cb.value, export_gdocs_cb.value, dry_run_cb.value
dl.skip_verify = skip_verify_cb.value

if not dest_text.value.strip():
    print('❌ Chưa nhập folder đích!')
elif not source_text.value.strip():
    print('❌ Chưa nhập folder nguồn!')
else:
    dl.run(dest_text.value.strip(), source_text.value.strip())


In [ ]:
#@title 3️⃣ 🗑️ Xóa checkpoint (copy lại từ đầu)
import os, re
from IPython.display import display, HTML
from google.colab import auth
from googleapiclient.discovery import build

display(HTML('''
<div style="background: linear-gradient(135deg, #dc3545 0%, #c82333 100%); padding: 12px 15px; border-radius: 8px; margin-bottom: 15px; color: white;">
    <b>🗑️ Xóa Checkpoint</b> - Reset để copy lại từ đầu
</div>
'''))

deleted = False

for f in ['/content/checkpoint.json', '/content/checkpoint.json.bak']:
    if os.path.exists(f):
        os.remove(f)
        print(f'   ✅ Đã xóa local: {f}')
        deleted = True

try:
    if dest_text.value.strip():
        auth.authenticate_user()
        svc = build('drive', 'v3')
        m = re.search(r'[-\w]{25,}', dest_text.value.strip())
        if m:
            did = m.group(0)
            r = svc.files().list(q=f"'{did}' in parents and name='.gdrive_turbo_checkpoint.json' and trashed=false",
                fields='files(id)', supportsAllDrives=True, pageSize=1).execute()
            if r and r.get('files'):
                svc.files().delete(fileId=r['files'][0]['id'], supportsAllDrives=True).execute()
                print(f'   ✅ Đã xóa checkpoint trên Drive')
                deleted = True
except Exception as e:
    print(f'   ⚠️ Không xóa được trên Drive: {str(e)[:50]}')

if not deleted:
    print('   ℹ️ Không có checkpoint nào.')

display(HTML('''
<div style="background: linear-gradient(135deg, #28a745 0%, #20c997 100%); padding: 12px 15px; border-radius: 8px; margin-top: 15px; color: white;">
    <b>🔄 Sẵn sàng!</b> Chạy lại <b>Cell 2</b> để copy từ đầu 🚀
</div>
'''))

In [ ]:
#@title 4️⃣ 📋 Xem Log & Lỗi
import os, json
from IPython.display import display, HTML

LOG_LINES = 100
log_file = '/content/gdrive_copy.log'
cp_file = '/content/checkpoint.json'

display(HTML('''
<div style="background: linear-gradient(135deg, #17a2b8 0%, #138496 100%); padding: 12px 15px; border-radius: 8px; margin-bottom: 15px; color: white;">
    <b>📋 Thông tin Log & Lỗi</b>
</div>
'''))

if os.path.exists(log_file):
    with open(log_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    display(HTML(f'<h4>📝 Log File ({len(lines):,} dòng)</h4>'))
    print('─' * 60)
    for line in lines[-LOG_LINES:]:
        print(line.rstrip())
    print('─' * 60)
    print(f'📊 Hiển thị {min(LOG_LINES, len(lines))}/{len(lines)} dòng cuối')
else:
    display(HTML('<div style="background: #e9ecef; padding: 10px; border-radius: 5px;">ℹ️ Chưa có log. Chạy Cell 2 trước.</div>'))

if os.path.exists(cp_file):
    with open(cp_file, 'r') as f:
        data = json.load(f)
    errors, shortcuts = data.get('e', []), data.get('shortcuts', [])

    display(HTML(f'''
    <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 15px 0; border-left: 4px solid #667eea;">
        <h4 style="margin: 0 0 10px 0;">📊 Thống kê Checkpoint</h4>
        <p style="margin: 5px 0;">✅ Đã copy: <b>{data.get('c', 0):,}</b> files</p>
        <p style="margin: 5px 0;">⏭️ Đã skip: <b>{data.get('s', 0):,}</b> files</p>
        <p style="margin: 5px 0;">📄 Đã export: <b>{data.get('exp', 0):,}</b> docs</p>
        <p style="margin: 5px 0;">💾 Tổng dung lượng: <b>{data.get('m', 0)/1024:.2f} GB</b></p>
        <p style="margin: 5px 0;">🕐 Cập nhật: <b>{data.get('t', 'N/A')[:19]}</b></p>
    </div>
    '''))

    if errors:
        display(HTML(f'<h4 style="color: #dc3545;">❌ {len(errors)} File Lỗi</h4>'))
        for i, ef in enumerate(errors[:10], 1):
            print(f"  {i}. 📄 {ef.get('name', '?')[:45]}")
            print(f"     📂 Path: {ef.get('path', 'N/A')[:40]}")
            print(f"     ⚠️ Lỗi: {ef.get('error', 'N/A')[:50]}\n")
        if len(errors) > 10:
            print(f'  ... và {len(errors)-10} file khác')
        display(HTML('<div style="background: #fff3cd; padding: 10px; border-radius: 5px; border-left: 4px solid #ffc107;">💡 <b>Tip:</b> Chạy lại Cell 2 để retry các file lỗi</div>'))
    else:
        display(HTML('<div style="background: #d4edda; padding: 10px; border-radius: 5px; border-left: 4px solid #28a745;">✅ <b>Tuyệt vời!</b> Không có file nào bị lỗi</div>'))

    if shortcuts:
        display(HTML(f'<h4>🔗 {len(shortcuts)} Shortcuts (đã bỏ qua)</h4>'))
        for i, sc in enumerate(shortcuts[:10], 1):
            print(f"  {i}. 🔗 {sc.get('name', '?')[:50]}")
        if len(shortcuts) > 10:
            print(f'  ... và {len(shortcuts)-10} shortcut khác')
else:
    if os.path.exists(log_file):
        display(HTML('<div style="background: #d4edda; padding: 10px; border-radius: 5px; border-left: 4px solid #28a745;">✅ <b>Copy hoàn tất!</b> Checkpoint đã được xóa tự động.</div>'))
    else:
        display(HTML('<div style="background: #e9ecef; padding: 10px; border-radius: 5px;">ℹ️ Chưa có checkpoint. Chạy Cell 2 trước.</div>'))

In [ ]:
#@title 5️⃣ 🔄 Retry Failed Files
import os, json
from IPython.display import display, HTML

display(HTML('''
<div style="background: linear-gradient(135deg, #fd7e14 0%, #dc3545 100%); padding: 12px 15px; border-radius: 8px; margin-bottom: 15px; color: white;">
    <b>🔄 Retry Failed Files</b> - Copy lại các file bị lỗi
</div>
'''))

cp_file = '/content/checkpoint.json'

if not os.path.exists(cp_file):
    print('ℹ️ Không có checkpoint. Chạy Cell 2 trước.')
else:
    with open(cp_file, 'r') as f:
        data = json.load(f)
    errors = data.get('e', [])

    if not errors:
        display(HTML('<div style="background: #d4edda; padding: 10px; border-radius: 5px;">✅ Không có file lỗi!</div>'))
    else:
        print(f'📋 Có {len(errors)} file lỗi:')
        for i, ef in enumerate(errors[:10], 1):
            print(f"   {i}. {ef.get('name', '?')[:50]}")
        if len(errors) > 10:
            print(f'   ... và {len(errors)-10} file khác')

        print('\n🔄 Đang reset để retry...')

        done_keys = set(data.get('k', []))
        removed = 0
        for ef in errors:
            fid = ef.get('id', '')
            keys_to_remove = [k for k in done_keys if fid in k]
            for k in keys_to_remove:
                done_keys.discard(k)
                removed += 1

        data['k'] = list(done_keys)
        data['e'] = []

        with open(cp_file, 'w') as f:
            json.dump(data, f)

        print(f'✅ Đã reset {removed} file')
        display(HTML('''
<div style="background: linear-gradient(135deg, #28a745 0%, #20c997 100%); padding: 12px 15px; border-radius: 8px; margin-top: 15px; color: white;">
    👉 Chạy lại <b>Cell 2</b> để retry!
</div>
'''))